<a href="https://colab.research.google.com/github/andreidm92/Agents_in_code/blob/main/practice/Lesson_12_Nurse_Student_Quiz_Maker.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


# Day 12 — Nurse: Student Quiz Maker (по протоколам медсестры)

## 🧠 Теория

### LlamaIndex: RetrievalQA Chain

**RetrievalQA** — это RAG-компонент, который:
- Выполняет поиск фрагментов из медицинских протоколов и инструкций по уходу.
- Генерирует ответы или тесты на основе найденных данных.

```python
from llama_index.core.query_engine import RetrievalQueryEngine

query_engine = RetrievalQueryEngine.from_args(
    retriever=retriever,
    llm=llm
)
response = query_engine.query("Составь 5 вопросов по протоколу внутривенного вливания (IV)")
```

---

### LangGraph: Checkpointing

Checkpointing позволяет:
- сохранять прогресс агента при генерации тестов или обучающих сценариев;
- восстанавливаться после ошибок или API-лимитов.

```python
graph = StateGraph(...)
graph.persisted("redis://localhost:6379/0")
```

---

### LangGraph: StateGraph Basics

Позволяет:
- строить сложные пайплайны — генерация вопросов → проверка качества → сохранение;
- включать циклы (например, если LLM сгенерировала слишком простой вопрос — повторить).

```python
from langgraph.graph import StateGraph, END

builder = StateGraph()
builder.add_node("generate_quiz", generate_quiz)
builder.set_entry_point("generate_quiz")
builder.add_edge("generate_quiz", END)

graph = builder.compile()
graph.invoke({})
```


In [1]:
# ✅ Рекомендуемая установка
!pip install -q -U llama-index==0.10.38 openai redis





     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.0/61.0 kB 3.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 720.4/720.4 kB 19.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 273.8/273.8 kB 17.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 49.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 44.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 141.9/141.9 kB 8.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.3/18.3 MB 81.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.8/295.8 kB 17.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 2.8 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
thinc 8.3.6 requires numpy<3.0.0,>=2.0.0, but you have numpy 1.26.4 which is incompatible.


In [1]:
from llama_index.core import VectorStoreIndex, SimpleDirectoryReader
from llama_index.core.node_parser import SentenceSplitter
from llama_index.core.storage.storage_context import StorageContext
from llama_index.core.vector_stores import SimpleVectorStore
from llama_index.embeddings.openai import OpenAIEmbedding
from llama_index.llms.openai import OpenAI


[nltk_data] Downloading package punkt_tab to
[nltk_data]     /usr/local/lib/python3.11/dist-
[nltk_data]     packages/llama_index/core/_static/nltk_cache...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.


In [2]:
import os, getpass
os.environ["OPENAI_API_KEY"] = getpass.getpass("Вставь OpenAI API ключ: ")

Вставь OpenAI API ключ: ··········


In [3]:
# 📚 Шаг 1: Загрузка и чтение лекций
# TODO: Загрузите свои PDF-файлы в папку 'lectures'
docs = SimpleDirectoryReader('lectures').load_data()

In [8]:
# 🔍 Шаг 2: Индексация с эмбеддингами
storage_context = StorageContext.from_defaults(vector_store=SimpleVectorStore())
index = VectorStoreIndex.from_documents(
    docs,
    embed_model=OpenAIEmbedding(),
    storage_context=storage_context
)


In [9]:
# 🧠 Шаг 3: Построение QueryEngine
llm = OpenAI(model="gpt-4")
query_engine = index.as_query_engine(llm=llm)



In [20]:
# ❓ Шаг 4: Пример генерации quiz
response = query_engine.query(
    'Составь quiz по общению в сестринском деле на русском языке '
)
print(response)

1. Какую позу тела следует использовать для проявления интереса к тому, что говорит пациент?
   а) Скрещенные руки и ноги
   б) Наклон вперед
   в) Отвернуться от пациента

2. Какие жесты можно использовать для поощрения разговора с пациентом?
   а) Кивание и улыбка
   б) Махание руками
   в) Пожатие плечами

3. Какие вопросы обычно самые полезные в общении с пациентом?
   а) Закрытые вопросы
   б) Открытые вопросы
   в) Риторические вопросы

4. Что следует делать с словами пациента в разговоре?
   а) Игнорировать их
   б) Отражать их, повторяя в нескольких словах
   в) Прерывать их

5. Что следует избегать в общении с пациентом?
   а) Частого употребления оценивающих слов
   б) Использования простого языка
   в) Поддержания контакта глазами

6. Как следует реагировать на то, что думает и чувствует пациент?
   а) Выражать согласие или несогласие
   б) Реагировать спокойно, не выражая согласия или несогласия
   в) Игнорировать его мысли и чувства

7. Что следует делать, если пациенту не

📌 Что делает graph.persisted(...)?
Когда ты вызываешь:

graph.persisted("redis://localhost:6379/0")
это говорит LangGraph:

* «Сохраняй всё состояние агента (включая переменные, шаги и промежуточные данные) в Redis».

При падении сервиса или повторном запуске — агент продолжит с того же места.

🏥 Почему это актуально в твоём проекте Nurse: Quiz Maker?
Если ты хочешь:

- генерировать длинные тесты

- обрабатывать несколько лекций

- или делать это через потоковое взаимодействие

…то Redis-чекпоинтинг поможет:

- избежать потерь при сбоях,

- разделить генерацию на несколько шагов,

- и вообще вести себя как серьёзный обучающий ассистент.

💡 Пример практического использования:

In [28]:
!pip install -q langgraph
!pip install -q langgraph-checkpoint-redis redis

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.8/58.8 kB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 272.8/272.8 kB 11.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.7/129.7 kB 8.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.0/46.0 kB 3.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.8/86.8 kB 6.7 MB/s eta 0:00:00


In [31]:
from langgraph.graph import StateGraph, END
from langgraph.checkpoint.redis import RedisSaver
from typing import TypedDict

class QuizState(TypedDict):
    topic: str
    query_engine: any
    quiz: str
    done: bool

def generate_quiz_node(state: dict) -> dict:
    topic = state.get("topic", "общение в сестринском деле")
    prompt = f"Сгенерируй 3 вопроса по теме '{topic}' с вариантами ответов"
    response = state["query_engine"].query(prompt)
    return {"quiz": str(response), "done": True}

graph = StateGraph(state_schema=QuizState)
graph.add_node("generate", generate_quiz_node)
graph.set_entry_point("generate")
graph.add_edge("generate", END)

# ✅ Используем контекстный менеджер

compiled = graph.compile()
compiled.invoke({
    "query_engine": query_engine,
    "topic": "общение в сестринском деле"
})

# with RedisSaver.from_conn_string("redis://localhost:6379") as checkpointer:
#     compiled = graph.compile(checkpointer=checkpointer)
#     compiled.invoke({
#         "query_engine": query_engine,
#         "topic": "общение в сестринском деле"
#     })


{'topic': 'общение в сестринском деле',
 'query_engine': <llama_index.core.query_engine.retriever_query_engine.RetrieverQueryEngine at 0x7ae2c4ac6050>,
 'quiz': '1. Какая поза тела рекомендуется при общении с пациентом в сестринском деле?\n   a) Скрещенные руки и ноги\n   b) Открытая поза, носки ног направлены к пациенту\n   c) Поза с отвернутым от пациента телом\n   d) Поза с закрытыми глазами\n\n2. Какие вопросы считаются наиболее полезными при общении с пациентом?\n   a) Закрытые вопросы, на которые можно ответить "да" или "нет"\n   b) Вопросы, начинающиеся с оценивающих слов\n   c) Открытые вопросы, начинающиеся с вопросительных слов\n   d) Вопросы, требующие только односложного ответа\n\n3. Как следует реагировать на мысли и чувства пациента в сестринском деле?\n   a) Выражать согласие или несогласие\n   b) Реагировать спокойно, не выражая согласия или несогласия\n   c) Игнорировать их\n   d) Критиковать их',
 'done': True}

**StateGraph — основная структура LangGraph, граф состояний.**


END — специальная метка для финального узла.

RedisSaver — механизм сохранения состояния в Redis (не используется в Colab, но пригодится в продакшене).

TypedDict — типизация для состояния графа.

📋 Определение состояния:

class QuizState(TypedDict):
    topic: str
    query_engine: any
    quiz: str
    done: bool

Ты явно описал, какие поля присутствуют в состоянии агента (state) на каждом шаге:

тема теста,

объект query_engine,

сгенерированный текст quiz,

флаг готовности done.


🧠 Узел графа:


def generate_quiz_node(state: dict) -> dict:
    topic = state.get("topic", "общение в сестринском деле")
    prompt = f"Сгенерируй 3 вопроса по теме '{topic}' с вариантами ответов"
    response = state["query_engine"].query(prompt)
    return {"quiz": str(response), "done": True}

Функция-узел, которая:

Берёт тему из state

Генерирует промпт

Запускает query_engine.query(...)

Возвращает результат в quiz и помечает, что задача выполнена (done: True)

🔁 Построение графа:


graph = StateGraph(state_schema=QuizState)
graph.add_node("generate", generate_quiz_node)
graph.set_entry_point("generate")
graph.add_edge("generate", END)


Задаётся граф с состоянием QuizState.

Добавляется один узел "generate".

Он — точка входа (set_entry_point).

После выполнения — переход в END.


🏁 Выполнение:


compiled = graph.compile()
compiled.invoke({
    "query_engine": query_engine,
    "topic": "общение в сестринском деле"
})

compile() превращает описание графа в исполняемый агент.

invoke(...) запускает его, передавая начальное состояние.

💾 (опционально) Чекпоинтинг:

with RedisSaver.from_conn_string(...) as checkpointer:
    compiled = graph.compile(checkpointer=checkpointer)

Если ты хочешь сохранять состояние (например, на сервере), ты бы подключил RedisSaver.

📌 Если хочешь Redis:
Зарегистрируйся на Redis Cloud → создай бесплатный Redis инстанс → получишь URL вроде redis://<username>:<password>@<host>:<port>.

Вставь его вместо localhost:6379.

🔁 Этот паттерн — отличная заготовка для более сложных агентов: можно добавлять шаги (генерация, валидация, экспорт), циклы, обработку ошибок и мн.др.